In [1]:
import os

import pandas as pd
import numpy as np
import scipy 

from MAGNN_preprocess_utils.preprocess import (
    get_column, 
    assign_index, 
    map_index_to_relation_file, 
    export_index2dat, 
    split_date,
    save_split_data2npz,
    get_metapath_adjacency_matrix
)

# Input data preprocess

In [2]:
# list all file paths for the original relation .dat files
file_path = os.getcwd()
# file 1, 2: microbe-disease
file1 = os.path.join(file_path, "../data", "MAGNN_data", "disbiome_taxid_mondo.dat")
file2 = os.path.join(file_path, "../data", "MAGNN_data", "gmmad2_taxid_mondo.dat")
# file 3, 4: microbe-metabolite
file3 = os.path.join(file_path, "../data", "MAGNN_data", "gmmad2_taxid_met.dat")
file4 = os.path.join(file_path, "../data", "MAGNN_data", "hmdb_taxid_met.dat")
# file 5: metabolite-disease
file5 = os.path.join(file_path, "../data", "MAGNN_data", "hmdb_met_disease.dat")

In [3]:
# get entity columns individually
microbes1 = get_column(file1, colname1="Microbe", colname2="Disease", col="col1")
microbes2 = get_column(file2, colname1="Microbe", colname2="Disease", col="col1")
microbes3 = get_column(file3, colname1="Microbe", colname2="Metabolite", col="col1")
microbes4 = get_column(file4, colname1="Microbe", colname2="Metabolite", col="col1")
all_microbes = assign_index([microbes1, microbes2, microbes3, microbes4])

disease1 = get_column(file1, colname1="Microbe", colname2="Disease", col="col2")
disease2 = get_column(file2, colname1="Microbe", colname2="Disease", col="col2")
disease3 = get_column(file5, colname1="Metabolite", colname2="Disease", col="col2")
all_diseases = assign_index([disease1, disease2, disease3])

metabolite1 = get_column(file3, colname1="Microbe", colname2="Metabolite", col="col2")
metabolite2 = get_column(file4, colname1="Microbe", colname2="Metabolite", col="col2")
metabolite3 = get_column(file5, colname1="Metabolite", colname2="Disease", col="col1")
all_metabolites = assign_index([metabolite1, metabolite2, metabolite3])

In [4]:
# export index file to MKG_data_processing/MAGNN/data
export_index2dat(all_microbes, "data/microbe_index.dat")
export_index2dat(all_metabolites, "data/metabolite_index.dat")
export_index2dat(all_diseases, "data/disease_index.dat")

In [5]:
# merge two relation dfs together
md_merged_df = map_index_to_relation_file([file1, file2], "Microbe", "Disease", all_microbes, all_diseases)
mm_df = map_index_to_relation_file([file3, file4], "Microbe", "Metabolite", all_microbes, all_metabolites)
metd_df = map_index_to_relation_file([file5], "Metabolite", "Disease", all_metabolites, all_diseases)

In [6]:
# export relational dfs to .dat files
export_index2dat(md_merged_df, "data/microbe_disease_idx.dat")
export_index2dat(mm_df, "data/microbe_metabolite_idx.dat")
export_index2dat(metd_df, "data/metabolite_disease_idx.dat")

# Create adjacency matrix

In [7]:
save_prefix = "data/preprocessed/"

In [8]:
microbe_disease = pd.read_csv("data/microbe_disease_idx.dat", encoding='utf-8', delimiter='\t', names=['MicrobeIdx', 'DiseaseIdx'])
microbe_metabolite = pd.read_csv('data/microbe_metabolite_idx.dat', encoding='utf-8', delimiter='\t', names=['MicrobeIdx', 'MetaboliteIdx'])
metabolite_disease = pd.read_csv('data/metabolite_disease_idx.dat', encoding='utf-8', delimiter='\t', names=['MetaboliteIdx', 'DiseaseIdx'])
num_microbe = 8202
num_metabolite = 23823
num_disease = 898

### Check for duplicate relationships in the data
If duplicates exist, need to make increments when creating adjacency matrix 
<br>
Microbe-Disease and Microbe-Metabolite relationships have duplicates

In [9]:
# Check for duplicates in microbe_disease data
# if duplicates=True, need to make increments when creating adjM 
duplicates_in_microbe_disease = microbe_disease[microbe_disease.duplicated()]
print(duplicates_in_microbe_disease)

# Check for duplicates in microbe_metabolite data
duplicates_in_microbe_metabolite = microbe_metabolite[microbe_metabolite.duplicated()]
print(duplicates_in_microbe_metabolite)

# Check for duplicates in metabolite_disease data
duplicates_in_metabolite_disease = metabolite_disease[metabolite_disease.duplicated()]
print(duplicates_in_metabolite_disease)

        MicrobeIdx  DiseaseIdx
5083            62          48
5278            10         115
5504            92           8
7773           153          48
8396           498         187
...            ...         ...
501973         221          94
503661         182          40
504745           7          94
505352         247          29
505647         255           8

[527 rows x 2 columns]
        MicrobeIdx  MetaboliteIdx
598604          41            586
598635          41             45
598671          41            720
598714          41            772
598737          41            795
598774          41            675
598807          41            734
598833          41            125
598844          41            116
598854          41            600
598855          49           1412
598858          41            988
598895         177            309
598913          41            260
598921         177            118
598943          41            976
598995          41        

# Split data into training, validation, and testing sets

In [10]:
md_train, md_val, md_test = split_date(microbe_disease, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1)
# save_split_data2npz(md_train, md_val, md_test, "data/micro_disease_train_val_test_idx.npz")

In [11]:
# training: 70%, validation: 20%, testing: 10%
train_val_test_idx = np.load("data/micro_disease_train_val_test_idx.npz")
train_idx = train_val_test_idx['train']
val_idx = train_val_test_idx['val']
test_idx = train_val_test_idx['test']

# reset microbe-disease index 
microbe_disease = microbe_disease.loc[train_idx].reset_index(drop=True)
microbe_disease.head()
print(f"Length of Training data: {len(microbe_disease)}")

Length of Training data: 354096


In [12]:
# build adjacency matrix
# 0 for microbe, 1 for disease, 2 for metabolite
dim = num_microbe + num_disease + num_metabolite

type_mask = np.zeros(dim, dtype=int)
type_mask[num_microbe:num_microbe+num_disease] = 1
type_mask[num_microbe+num_disease:]=2

adjM = np.zeros((dim, dim), dtype=int)
for _, row in microbe_disease.iterrows():
    microID = row["MicrobeIdx"]
    diseaseID = num_microbe + row["DiseaseIdx"]
    # increment accounts for multiple links exist between same microbe and disease relationships
    adjM[microID, diseaseID] += 1
    adjM[diseaseID, microID] += 1
for _, row in microbe_metabolite.iterrows():
    microID = row["MicrobeIdx"]
    metID = num_microbe + num_disease + row["MetaboliteIdx"]
    adjM[microID, metID] += 1
    adjM[metID, microID] += 1
for _, row in metabolite_disease.iterrows():
    metID = num_microbe + num_disease + row["MetaboliteIdx"]
    diseaseID = num_microbe + row["DiseaseIdx"]
    adjM[metID, diseaseID] = 1
    adjM[diseaseID, metID] = 1

#### Check for non-zero entries in the adjacency matrix

In [13]:
# example of non-zero entries in the adjacency matrix
adjM[8202:8212, 0:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 1, 0, 1, 1, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]])

## Metapath Analysis

In [28]:
# TODO: Ways to summarize and compare the non-zero metapath adjacency matrix?

### Total path count analysis

In [29]:
expected_metapath_l3 = [
    [0, 1, 0],
    [0, 1, 2],
    [0, 2, 0],
    [0, 2, 1],
    [1, 0, 1],
    [1, 0, 2],
    [1, 2, 0],
    [1, 2, 1],
    [2, 0, 1],
    [2, 0, 2],
    [2, 1, 0],
    [2, 1, 2]
]

metapath_strengths_l = []

for metapath in expected_metapath_l3:
    metapath_adjM = get_metapath_adjacency_matrix(adjM, type_mask, metapath)
    
    print(metapath_adjM.sum())
    non_zero_values = metapath_adjM[metapath_adjM > 0]
    max_strength = non_zero_values.max() if non_zero_values.size > 0 else 0
    min_strength = non_zero_values.min() if non_zero_values.size > 0 else 0
    mean_strength = non_zero_values.mean() if non_zero_values.size > 0 else 0
    
    metapath_strengths = {
        "max": max_strength,
        "min": min_strength,
        "mean": mean_strength
    }
    metapath_strengths_l.append(metapath_strengths)
    metapath_df = pd.DataFrame(metapath_strengths_l)

1511417774
3130452
384891158
1497403
20477412
2037517
1497403
92218
2037517
437895926
3130452
402996188


In [27]:
metapath_df

,max,min,mean
0,127,1,33.701239
1,10,1,1.331122
2,1011,1,371.274302
3,159,1,4.813284
4,4555,1,346.628275
5,48,1,14.502210
6,159,1,4.813284
7,20017,1,3.185313
8,48,1,14.502210
9,906,1,199.866325


In [22]:
expected_metapath_l4 = [
    [0, 1, 0, 1],
    [0, 1, 0, 2],
    [0, 1, 2, 0],
    [0, 1, 2, 1],
    [0, 2, 0, 1],
    [0, 2, 0, 2],
    [0, 2, 1, 0],
    [0, 2, 1, 2],
    [1, 0, 1, 0],
    [1, 0, 1, 2],
    [1, 0, 2, 0],
    [1, 0, 2, 1],
    [1, 2, 0, 1],
    [1, 2, 0, 2],
    [1, 2, 1, 0],
    [1, 2, 1, 2],
    [2, 1, 0, 1],
    [2, 1, 0, 2],
    [2, 1, 2, 0],
    [2, 1, 2, 1]
]

metapath_strengths_l = []

for metapath in expected_metapath_l4:
    metapath_adjM = get_metapath_adjacency_matrix(adjM, type_mask, metapath)
    
    non_zero_values = metapath_adjM[metapath_adjM > 0]
    max_strength = non_zero_values.max() if non_zero_values.size > 0 else 0
    min_strength = non_zero_values.min() if non_zero_values.size > 0 else 0
    mean_strength = non_zero_values.mean() if non_zero_values.size > 0 else 0
    
    metapath_strengths = {
        "metapath": metapath,
        "max": max_strength,
        "min": min_strength,
        "mean": mean_strength
    }
    
    metapath_strengths_l.append(metapath_strengths)

metapath_df = pd.DataFrame(metapath_strengths_l)

In [23]:
metapath_df

,metapath,max,min,mean
0,"[0, 1, 0, 1]",220828,1,38881.417479
1,"[0, 1, 0, 2]",2882,1,866.685620
2,"[0, 1, 2, 0]",567,1,99.787564
3,"[0, 1, 2, 1]",1372,1,12.768375
4,"[0, 2, 0, 1]",26202,1,5770.010543
5,"[0, 2, 0, 2]",562988,1,143093.646563
6,"[0, 2, 1, 0]",567,1,99.787564
7,"[0, 2, 1, 2]",1250,1,11.798923
8,"[1, 0, 1, 0]",220828,1,38881.417479
9,"[1, 0, 1, 2]",19757,1,527.937355


# Continue for adjacency Matrix

In [15]:
# map each microbe to a list of diseases
microbe_disease_list = {i: adjM[i, num_microbe:num_microbe+num_disease].nonzero()[0] for i in range(num_microbe)}
# map each disease to a list of microbes
disease_microbe_list = {i: adjM[num_microbe+i, :num_microbe].nonzero()[0] for i in range(num_disease)}
# map each metabolite to a list of diseases
metabolite_disease_list = {i: adjM[num_microbe+num_disease+i, num_microbe:num_microbe+num_disease].nonzero()[0] for i in range(num_metabolite)}
# map each disease to a list of metabolites
disease_metabolite_list = {i: adjM[num_microbe+i, num_microbe+num_disease:num_microbe+num_disease+num_metabolite].nonzero()[0] for i in range(num_disease)}
# map each microbe to a list of metabolites
microbe_metabolite_list = {i: adjM[i, num_microbe+num_disease:num_microbe+num_disease+num_metabolite].nonzero()[0] for i in range(num_microbe)}
# map each metabolite to a list of microbes
metabolite_microbe_list = {i: adjM[num_microbe+num_disease+i, :num_microbe].nonzero()[0] for i in range(num_metabolite)}